In [2]:
#Libraries
!pip install deep-translator
!pip install tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

# Import functions for data preprocessing & data preparation
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.2 MB/s eta 0:00:00


In [3]:
data = pd.read_csv('input/pemiluyt.csv')
data.columns
data1=data.drop(['Timestamp','VideoID','Date'],axis=1)
data1

FileNotFoundError: [Errno 2] No such file or directory: 'input/pemiluyt.csv'

In [ ]:
from deep_translator import GoogleTranslator
from tqdm import tqdm
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Pastikan untuk mendownload lexicon VADER
nltk.download('vader_lexicon')

# Inisialisasi SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()

# Terjemahkan teks komentar ke bahasa Inggris dengan progress bar
data1['Comment_EN'] = [GoogleTranslator(source='id', target='en').translate(comment) if comment else "" for comment in tqdm(data1['Comment'], desc="Translating")]

# Isi None atau NaN dalam Comment_EN dengan string kosong untuk menghindari error
data1['Comment_EN'] = data1['Comment_EN'].fillna("")

# Analisis sentimen pada teks yang sudah diterjemahkan
data1["Positive"] = [sentiments.polarity_scores(i)["pos"] if i else 0 for i in data1["Comment_EN"]]
data1["Negative"] = [sentiments.polarity_scores(i)["neg"] if i else 0 for i in data1["Comment_EN"]]
data1["Neutral"] = [sentiments.polarity_scores(i)["neu"] if i else 0 for i in data1["Comment_EN"]]
data1['Compound'] = [sentiments.polarity_scores(i)["compound"] if i else 0 for i in data1["Comment_EN"]]

# Mengklasifikasikan hasil sentimen
score = data1["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05:
        sentiment.append('Positive')
    elif i <= -0.05:
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
data1["Sentiment"] = sentiment

# Lihat hasilnya
data1.head()


In [ ]:
data2=data1.drop(['Positive','Negative','Neutral','Compound'],axis=1)
data2.head()

In [ ]:
import nltk
nltk.download('stopwords')

stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer()
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

In [ ]:
def text_processing(text):
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)

    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)

    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)

    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # remove special characters from text
    text = re.sub(r'\W', ' ', text)

    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])

    # stemming using porter stemmer from nltk package - msh a7sn 7aga - momken: lancaster, snowball
    # text=' '.join([porter_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([lancaster_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([snowball_stemer.stem(word) for word in word_tokenize(text)])

    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


nltk.download('omw-1.4')
data_copy = data2.copy()
data_copy.Comment = data_copy.Comment.apply(lambda text: text_processing(text))

In [ ]:
processed_data = {
    'Sentence':data_copy.Comment,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

In [ ]:
processed_data['Sentiment'].value_counts()